## Panel - Interacting with the visual

In the [heat notebook](01_heat.ipynb) we generated a visual of the thresholded surface temperature over Philadelphia. This plot is rendered with bokeh and uses a running python kernel to resample the data when zooming in and out. But what if we want to allow the user to set their own threshold. To enable this kind of interactivity we'll use [panel](https://panel.pyviz.org).

In [ ]:
import intake
import pandas as pd

import holoviews as hv
from holoviews import opts
import hvplot.xarray
import hvplot.pandas

import panel as pn

from geoviews.tile_sources import EsriImagery

hv.extension('bokeh')

### Load the data

First load in the data that we computed and saved in the previous notebook. 

In [ ]:
mtf_source = intake.open_zarr('./data/mean_temp.zarr')
mean_temp_f = mtf_source.read()
mean_temp_f

In [ ]:
trees_df = pd.read_parquet('./data/trees.parq')
trees_df.head()

Make the same special colormap:

In [ ]:
import colorcet as cc
special_cmap = cc.fire[::-1][90:]

### Use panel.interact

Write a function that generates the thresholded plot and takes the temperature at which to threshold as an argument. Use the `pn.interact` decorator to "widgetize" the function. 

In [ ]:
@pn.interact(thresh=(70, 100))
def threshold(thresh=90):
    thresholded_temp_p = (mean_temp_f.where(mean_temp_f.temperature > thresh)
        .hvplot(x='x', y='y', z='temperature', cmap=special_cmap, geo=True, 
                height=500, width=500, colorbar=False, legend=False)
        .relabel(f'Mean Temp (F) > {thresh}')
        .redim(value='Temperature (F)'))

    return thresholded_temp_p.options(alpha=0.5) * EsriImagery

In [ ]:
threshold

There are a few limitations to this approach. For instance, on each interaction we: 
  1. Recompute the data 
  2. Re-render the whole plot
  2. Call out to the python kernel (doesn't work in a static version)
  
### Using color clipping

To combat some of these limitations we can use color clipping and some recently developed panel functionality to link interactions using js. 

We'll start by defining the plot.

In [ ]:
p = (mean_temp_f.hvplot(x='x', y='y', geo=True, height=500, width=550, 
                        cmap=special_cmap, alpha=.5, legend=False)
                .relabel('Mean Surface Temp (F)'))

Next we'll set up a clipped colormap that only shows values between an upper and lower bound.

In [ ]:
options = opts.Image(clipping_colors={'min': 'transparent', 'max': 'transparent'}, 
                     cmap=special_cmap)
clipped = p.opts(options, clone=True)
clipped.redim.range(temperature=(90, 120)) * EsriImagery

So we've solved the problem of the data being recomputed every time. Now we can add in interaction. 

### Using JS linking

Now we are trying to interact with things that are on the javascript side. So instead of having the interaction happen on the python side, we can have it on the js side. This will make it faster and also allow it to happen *without a python kernel running*.

In [ ]:
start, end = (int(mean_temp_f.temperature.min() - 1), 
              int(mean_temp_f.temperature.max() + 1))

thresh_slider = pn.widgets.RangeSlider(start=start, end=end, value=(start, end), 
                                       step=1, bar_color='#ff0000',
                                       name='Temperature range')

thresh_slider.jslink(clipped, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
"""})

pn.Row(thresh_slider, clipped * EsriImagery)

### Adding in trees

It is hypothesized that where there are more trees the land surface temperature will be less extreme. To explore this, we will overlay street trees with the thresholded land surface temperature:

In [ ]:
trees_p = trees_df.hvplot.points('Longitude', 'Latitude', 
                                geo=True, datashade=True, 
                                height=500, width=500, 
                                dynspread=True, cmap=cc.kgy[::-1],
                                legend=False).relabel('Street Tree Density')

### Turn tree layer on and off

It'd be nice to be able to turn the tree layer on and off using a toggle button.

In [ ]:
trees_toggle = pn.widgets.Toggle(name='Tree Layer', value=True)
trees_toggle.jslink(trees_p, value='glyph_renderer.visible')

pn.Row(trees_toggle, trees_p)

### Adding some more interaction

To round out the dashboard, we can add a base map selector, an temperature alpha slider, a tree alpha slider... 

In [ ]:
from holoviews.streams import Params

import geoviews as gv
from geoviews.tile_sources import tile_sources

In [ ]:
temp_alpha_slider = pn.widgets.FloatSlider(
    start=0, end=1, value=.5, name='Temperature alpha')
temp_alpha_slider.jslink(clipped, value='glyph.global_alpha')

trees_alpha_slider = pn.widgets.FloatSlider(
    start=0, end=1, value=.7, name='Trees alpha')
trees_alpha_slider.jslink(trees_p, value='glyph.global_alpha')

basemap_selector = pn.widgets.Select(name='Tiling Service', value=EsriImagery, options=tile_sources)

def set_tile_source(value):
    return value
    
dmap = hv.DynamicMap(set_tile_source, streams=[Params(basemap_selector, ['value'])])

dashboard = pn.Row(
    pn.Column(
        '# Heat and Trees',
        thresh_slider, 
        temp_alpha_slider,
        basemap_selector,
        trees_toggle,
        trees_alpha_slider),
    clipped * trees_p * dmap 
)

### Deploy

To deploy the dashboard just call the `.servable()` method on the panel object. Then use:

```
panel serve --show Heat_and_Trees_part_2.ipynb
```

In [ ]:
dashboard.servable()

### Embed

To embed the image in a static website, we need to run through all the python interactions and save off the possible states. Panel provides a mechanism to do this using `embed`. When you call this method, the plot will run through every combination of the options making some decisions about step to reduce the number of combinations. As an example we can first do a simple save:

In [ ]:
dashboard.save('only_js.html')

A lot of the dashboard works in that version. The only part that doesn't is the tile selector. For that we need a python kernel OR embedded output.

In [ ]:
dashboard.save('index.html', title='Heat and Trees', embed=True)